In [1]:
%%configure -f
{"driverMemory": "8G", 
 "executorMemory": "16G", 
 "executorCores": 2, 
 "numExecutors": 15,
 "name": "allocation-analysis",
 "heartbeatTimeoutInSecond": 18000,
 "conf": {"spark.dynamicAllocation.enabled":"false"}
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
88,application_1627924993397_0012,pyspark,idle,Link,Link,
101,None,pyspark,starting,,,


In [2]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
102,application_1627924993397_0026,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType
from datetime import date, timedelta, datetime

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1580,application_1600172976670_1235,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
k=2
col_id=55

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
run_date = date.today().strftime("%Y-%m-%d")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
rdate=datetime.strptime(run_date, '%Y-%m-%d')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
start_date=rdate-timedelta(days=30)
start_date_str = start_date.strftime("%Y-%m-%d")
start_date_str

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2020-10-17'

In [8]:
model_output_path='s3a://midgar-aws-workspace/prod/shinra/modelling/journeyautomation/tenant/paytm-india/scores/customerCombinedScores/'



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
output_path="s3a://midgar-aws-workspace/stg/shinra/telescope/analysis/push_allocation_metrics/"
topk_allocation_path=output_path+'/topk_allocation'
topk_allocation_path

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3a://midgar-aws-workspace/stg/shinra/telescope/analysis/push_allocation_metrics//topk_allocation'

In [10]:
# spark.read.parquet(model_output_path).select('customer_id', 'scores').printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- customer_id: long (nullable = true)
 |-- scores: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- l1_category_id: integer (nullable = true)
 |    |    |-- probability: double (nullable = true)

In [11]:
model_output=spark.read.parquet(model_output_path)\
    .filter(F.col('dt').between(start_date_str, rdate))\
    .select('customer_id', 'scores.l1_category_id', 'dt')\
    .withColumn("tops", F.slice("l1_category_id", 1, k))\
    .filter(F.array_contains(F.col('tops'), col_id))\
    .groupBy('dt')\
    .agg(F.count('customer_id').alias('customer_size'))\
    .withColumn('model_topk', F.lit(k))\
    .withColumn('cma_id', F.lit(col_id))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
model_output.coalesce(1).write.mode("overwrite").option("header","true").csv(topk_allocation_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
model_output=spark.read.option("header","true").csv(topk_allocation_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
model_output.show(30,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------+----------+------+
|dt        |customer_size|model_topk|cma_id|
+----------+-------------+----------+------+
|2020-11-11|455742       |2         |55    |
|2020-11-08|783484       |2         |55    |
|2020-11-12|457966       |2         |55    |
|2020-11-10|450463       |2         |55    |
|2020-11-03|647170       |2         |55    |
|2020-10-21|922390       |2         |55    |
|2020-11-07|749392       |2         |55    |
|2020-10-20|934526       |2         |55    |
|2020-11-05|674103       |2         |55    |
|2020-10-28|770082       |2         |55    |
|2020-11-02|650422       |2         |55    |
|2020-11-04|666259       |2         |55    |
|2020-10-22|824636       |2         |55    |
|2020-10-17|928328       |2         |55    |
|2020-10-26|785211       |2         |55    |
|2020-10-18|934392       |2         |55    |
|2020-10-25|795610       |2         |55    |
|2020-11-13|466715       |2         |55    |
|2020-11-14|467179       |2         |55    |
|2020-10-3